# 💰 Personal Finance Monte Carlo Simulation
> A robust, production-ready financial planning model that simulates future savings trajectories and estimates financial risk using Monte Carlo methods.

In [ ]:
# ============================================================
# 📦 CELL 1 — Install & Import Libraries
# ============================================================
# Uncomment the line below if running on Google Colab
# from google.colab import drive; drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Global plot style
sns.set_theme(style='darkgrid', palette='muted')
plt.rcParams['figure.dpi'] = 120
print("✅ Libraries loaded successfully.")


## 📂 Step 1 — Load & Inspect Data

In [ ]:
# ============================================================
# 📂 CELL 2 — Load Data
# ============================================================
# 🔧 ADJUST PATH as needed
DATA_PATH = '/content/drive/MyDrive/Finace Data/data.csv'

df = pd.read_csv(DATA_PATH)

print(f"📊 Dataset shape: {df.shape}")
print(f"\n🔍 Columns: {list(df.columns)}")
print("\n📋 Data Types & Nulls:")
print(df.info())
df.head()


In [ ]:
# ============================================================
# 🧹 CELL 3 — Data Quality Report
# ============================================================
print("=" * 55)
print("📋 DESCRIPTIVE STATISTICS")
print("=" * 55)
print(df.describe().round(2).to_string())

print("\n" + "=" * 55)
print("🚨 MISSING VALUES")
print("=" * 55)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_report = pd.DataFrame({'Missing': missing, 'Pct(%)': missing_pct})
print(missing_report[missing_report['Missing'] > 0].to_string() or "  ✅ No missing values!")

print("\n" + "=" * 55)
print("♻️  DUPLICATE ROWS")
print("=" * 55)
print(f"  Duplicate rows: {df.duplicated().sum()}")


## 📊 Step 2 — Exploratory Data Analysis (EDA)

In [ ]:
# ============================================================
# 📊 CELL 4 — Distribution Plots (Histograms)
# ============================================================
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

n_cols = 4
n_rows = int(np.ceil(len(numeric_cols) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.histplot(df[col].dropna(), kde=True, bins=30, ax=axes[i], color='steelblue')
    axes[i].set_title(col, fontsize=12, fontweight='bold')
    axes[i].set_xlabel('')

# Hide unused subplots
for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle("📊 Distribution of Numerical Features", fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 📦 CELL 5 — Boxplots (Outlier Detection)
# ============================================================
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.boxplot(x=df[col].dropna(), ax=axes[i], color='lightcoral')
    axes[i].set_title(col, fontsize=12, fontweight='bold')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.suptitle("📦 Boxplots — Outlier Detection", fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 🔥 CELL 6 — Correlation Heatmap
# ============================================================
plt.figure(figsize=(14, 10))
corr = df[numeric_cols].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))  # show lower triangle only
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm',
            mask=mask, vmin=-1, vmax=1,
            linewidths=0.5, square=True)
plt.title("🔥 Correlation Matrix (Lower Triangle)", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## 🧹 Step 3 — Feature Engineering & Selection

In [ ]:
# ============================================================
# 🧹 CELL 7 — Clean & Engineer Features
# ============================================================
# Drop non-financial ID columns
drop_cols = ['ID', 'Customer_ID', 'Name', 'SSN', 'Month', 'Type_of_Loan']
df_clean = df.drop(columns=[c for c in drop_cols if c in df.columns])

# --- Feature Engineering ---
# Actual monthly savings from desired percentage
df_clean['Monthly_Savings'] = df_clean['Income'] * (df_clean['Desired_Savings_Percentage'] / 100)

# Lifestyle cost (configurable — default 30%)
LIFESTYLE_PCT = 0.30
df_clean['Lifestyle_Expenses'] = df_clean['Income'] * LIFESTYLE_PCT

# Total monthly expenses (all known costs)
expense_cols_available = [c for c in ['Loan_Repayment', 'Education_Expenses', 'Healthcare_Expenses'] 
                          if c in df_clean.columns]
df_clean['Total_Expenses'] = df_clean[expense_cols_available].sum(axis=1) + df_clean['Lifestyle_Expenses']

# Net monthly cash flow
df_clean['Net_Cash_Flow'] = df_clean['Income'] - df_clean['Total_Expenses']

# Savings Rate (actual)
df_clean['Savings_Rate'] = df_clean['Net_Cash_Flow'] / df_clean['Income'].replace(0, np.nan)

# Debt-to-Income Ratio
if 'Loan_Repayment' in df_clean.columns:
    df_clean['DTI_Ratio'] = df_clean['Loan_Repayment'] / df_clean['Income'].replace(0, np.nan)

print("✅ Engineered features added:")
new_cols = ['Monthly_Savings', 'Lifestyle_Expenses', 'Total_Expenses', 
            'Net_Cash_Flow', 'Savings_Rate', 'DTI_Ratio']
for c in new_cols:
    if c in df_clean.columns:
        print(f"  • {c}: mean={df_clean[c].mean():.2f}, std={df_clean[c].std():.2f}")


In [ ]:
# ============================================================
# 🔬 CELL 8 — Feature Selection for Monte Carlo
# ============================================================
numeric_df = df_clean.select_dtypes(include=['float64', 'int64']).dropna()

# Step A: Variance threshold
selector = VarianceThreshold(threshold=0.01)
selector.fit(numeric_df)
selected = numeric_df.columns[selector.get_support()]
print(f"✅ After variance filter: {len(selected)} features")

# Step B: Remove highly correlated features (threshold 0.85)
corr_matrix = numeric_df[selected].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.85)]
monte_carlo_features = [f for f in selected if f not in to_drop]

print(f"✅ After correlation filter (>0.85 dropped): {len(monte_carlo_features)} features")
print("\n📌 Final Monte Carlo Features:")
for f in monte_carlo_features:
    print(f"  • {f}")

df_monte = numeric_df[monte_carlo_features]


## 🎲 Step 4 — Monte Carlo Simulation

In [ ]:
# ============================================================
# 🎲 CELL 9 — Monte Carlo Setup
# ============================================================
np.random.seed(42)  # reproducibility

# ---- Configuration ----
N_SIMULATIONS  = 10_000   # number of simulation paths
N_MONTHS       = 24       # simulation horizon (months)
CRISIS_THRESHOLD = 0      # savings below this = financial crisis

# ---- Extract simulation parameters from data ----
income_mean  = df_clean['Income'].mean()
income_std   = df_clean['Income'].std()
income_lower = df_clean['Income'].quantile(0.01)   # floor: prevent extreme negatives

expense_mean = df_clean['Total_Expenses'].mean()
expense_std  = df_clean['Total_Expenses'].std()

# Initial savings: use individual starting point per simulation path
initial_savings_pool = df_clean['Monthly_Savings'].values

print("=" * 45)
print("🎲 MONTE CARLO CONFIGURATION")
print("=" * 45)
print(f"  Simulations  : {N_SIMULATIONS:,}")
print(f"  Horizon      : {N_MONTHS} months")
print(f"  Income μ     : ${income_mean:,.0f}")
print(f"  Income σ     : ${income_std:,.0f}")
print(f"  Expense μ    : ${expense_mean:,.0f}")
print(f"  Expense σ    : ${expense_std:,.0f}")


In [ ]:
# ============================================================
# 🚀 CELL 10 — Run Monte Carlo Simulation
# ============================================================
future_savings = np.zeros((N_SIMULATIONS, N_MONTHS))

for i in range(N_SIMULATIONS):
    # Each path starts from a randomly sampled individual's savings
    savings = np.random.choice(initial_savings_pool)

    for t in range(N_MONTHS):
        # Stochastic income (clipped to avoid extreme negatives)
        income = max(np.random.normal(income_mean, income_std), income_lower)

        # Stochastic expenses (clipped to be non-negative)
        expenses = max(np.random.normal(expense_mean, expense_std), 0)

        # Optional: apply 0.3% monthly income growth trend
        income *= (1 + 0.003) ** t

        savings += (income - expenses)
        future_savings[i, t] = savings

print(f"✅ Simulation complete: {N_SIMULATIONS:,} paths × {N_MONTHS} months")


## 📈 Step 5 — Results & Visualisation

In [ ]:
# ============================================================
# 📈 CELL 11 — Simulation Paths Plot
# ============================================================
fig, ax = plt.subplots(figsize=(14, 6))

# Plot sample paths
sample_paths = 300
for i in range(sample_paths):
    ax.plot(future_savings[i], alpha=0.08, linewidth=0.7, color='steelblue')

# Percentile bands
p5  = np.percentile(future_savings, 5,  axis=0)
p25 = np.percentile(future_savings, 25, axis=0)
p50 = np.percentile(future_savings, 50, axis=0)
p75 = np.percentile(future_savings, 75, axis=0)
p95 = np.percentile(future_savings, 95, axis=0)
months = np.arange(1, N_MONTHS + 1)

ax.fill_between(months, p5,  p95, alpha=0.15, color='orange', label='5th–95th pct')
ax.fill_between(months, p25, p75, alpha=0.25, color='orange', label='25th–75th pct')
ax.plot(months, p50, color='red', linewidth=2.5, label='Median (50th pct)')
ax.plot(months, p5,  color='darkorange', linewidth=1.2, linestyle='--', label='5th pct')
ax.plot(months, p95, color='darkorange', linewidth=1.2, linestyle='--', label='95th pct')

ax.axhline(0, color='black', linestyle=':', linewidth=1.5, label='Crisis Line ($0)')
ax.set_xlabel("Month", fontsize=12)
ax.set_ylabel("Cumulative Savings ($)", fontsize=12)
ax.set_title(f"🎲 Monte Carlo Simulation — {N_SIMULATIONS:,} Paths over {N_MONTHS} Months",
             fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=9)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x:,.0f}'))
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 📊 CELL 12 — Final Savings Distribution
# ============================================================
final_savings = future_savings[:, -1]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Histogram
sns.histplot(final_savings, bins=80, kde=True, ax=axes[0], color='steelblue')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2, label='Crisis Line')
axes[0].axvline(np.median(final_savings), color='green', linestyle='-', linewidth=2,
                label=f'Median: ${np.median(final_savings):,.0f}')
axes[0].set_title(f"📊 Final Savings Distribution (Month {N_MONTHS})", fontweight='bold')
axes[0].set_xlabel("Savings ($)")
axes[0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x/1e3:.0f}K'))
axes[0].legend()

# Percentile summary bar chart
pcts = [5, 10, 25, 50, 75, 90, 95]
vals = [np.percentile(final_savings, p) for p in pcts]
colors = ['#d32f2f' if v < 0 else '#388e3c' for v in vals]
bars = axes[1].bar([f"P{p}" for p in pcts], vals, color=colors, edgecolor='white')
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_title("📋 Savings Percentiles", fontweight='bold')
axes[1].set_ylabel("Savings ($)")
axes[1].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'${x/1e3:.0f}K'))
for bar, val in zip(bars, vals):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(vals)*0.01,
                 f'${val/1e3:.0f}K', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 📋 CELL 13 — Risk & Summary Report
# ============================================================
crisis_prob = np.mean(final_savings < CRISIS_THRESHOLD)
p5_val      = np.percentile(final_savings, 5)
median_val  = np.median(final_savings)
p95_val     = np.percentile(final_savings, 95)
pct_positive = np.mean(final_savings > 0) * 100

print("=" * 55)
print("📋  MONTE CARLO FINANCIAL RISK REPORT")
print("=" * 55)
print(f"  Simulations        : {N_SIMULATIONS:,}")
print(f"  Time Horizon       : {N_MONTHS} months ({N_MONTHS//12} years)")
print(f"  Crisis Threshold   : ${CRISIS_THRESHOLD:,.0f}")
print("-" * 55)
print(f"  🔴 Crisis Probability  : {crisis_prob:.4%}")
print(f"  🟢 Positive Savings    : {pct_positive:.1f}% of paths")
print(f"  📉 5th Percentile      : ${p5_val:,.0f}")
print(f"  📊 Median Outcome      : ${median_val:,.0f}")
print(f"  📈 95th Percentile     : ${p95_val:,.0f}")
print("-" * 55)

# Risk classification
if crisis_prob < 0.01:
    risk_level = "🟢 LOW RISK"
elif crisis_prob < 0.05:
    risk_level = "🟡 MODERATE RISK"
elif crisis_prob < 0.15:
    risk_level = "🟠 HIGH RISK"
else:
    risk_level = "🔴 CRITICAL RISK"

print(f"  ⚠️  Risk Classification : {risk_level}")
print("=" * 55)

# Sensitivity summary
print("\n📌 INTERPRETATION:")
if crisis_prob < 0.01:
    print("  The population shows strong financial resilience.")
    print(f"  Only {crisis_prob:.2%} of simulations result in a financial crisis.")
else:
    print(f"  {crisis_prob:.2%} of simulations result in negative savings.")
    print("  Consider reviewing expense management or income diversification.")


In [ ]:
# ============================================================
# 🔍 CELL 14 — Sensitivity Analysis (Income & Expense Shocks)
# ============================================================
scenarios = {
    'Base Case':        (0.0,  0.0),
    '+10% Income':      (0.10, 0.0),
    '-10% Income':      (-0.10, 0.0),
    '+20% Expenses':    (0.0,  0.20),
    '-20% Expenses':    (0.0,  -0.20),
    'Recession':        (-0.15, 0.20),   # income drops, expenses rise
}

results = {}
for scenario, (income_shock, expense_shock) in scenarios.items():
    s_income_mean  = income_mean  * (1 + income_shock)
    s_expense_mean = expense_mean * (1 + expense_shock)

    sim = np.zeros(N_SIMULATIONS)
    for i in range(N_SIMULATIONS):
        savings = np.random.choice(initial_savings_pool)
        for t in range(N_MONTHS):
            inc = max(np.random.normal(s_income_mean, income_std), 0)
            exp = max(np.random.normal(s_expense_mean, expense_std), 0)
            savings += (inc - exp)
        sim[i] = savings

    results[scenario] = {
        'crisis_prob': np.mean(sim < 0),
        'median':      np.median(sim),
        'p5':          np.percentile(sim, 5),
    }

print("=" * 70)
print(f"{'Scenario':<22} {'Crisis Prob':>12} {'Median ($)':>14} {'P5 ($)':>12}")
print("-" * 70)
for scenario, r in results.items():
    print(f"{scenario:<22} {r['crisis_prob']:>12.3%} {r['median']:>14,.0f} {r['p5']:>12,.0f}")
print("=" * 70)

# Bar chart of crisis probability by scenario
fig, ax = plt.subplots(figsize=(12, 5))
scenario_names = list(results.keys())
probs = [results[s]['crisis_prob'] * 100 for s in scenario_names]
colors = ['#d32f2f' if p > 5 else '#f57c00' if p > 1 else '#388e3c' for p in probs]
bars = ax.bar(scenario_names, probs, color=colors, edgecolor='white')
ax.set_ylabel("Crisis Probability (%)")
ax.set_title("🔍 Sensitivity Analysis — Crisis Probability by Scenario", fontweight='bold')
ax.axhline(5, color='red', linestyle='--', linewidth=1.2, label='5% threshold')
for bar, p in zip(bars, probs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
            f'{p:.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()
